## 1. Import Libraries

In [5]:
import random
import copy
import time


## 2. k-SAT Problem Generation

In [6]:
def generate_k_sat(k: int, m: int, n: int):
    clauses = []
    for _ in range(m):
        clause = set()
        while len(clause) < k:
            var = random.randint(1, n)
            if random.random() < 0.5:
                var = -var
            clause.add(var)
        clauses.append(list(clause))
    return clauses


## 3. Print Clauses

In [7]:
def print_clauses(clauses):
    for clause in clauses:
        print(" OR ".join([f"x{abs(lit)}" if lit > 0 else f"¬x{abs(lit)}" for lit in clause]))


## 4. Heuristic Functions

In [8]:
def heuristic_unsatisfied(clauses, assignment):
    unsat = 0
    for clause in clauses:
        satisfied = False
        for lit in clause:
            if lit > 0 and assignment[lit - 1] == 1:
                satisfied = True
            elif lit < 0 and assignment[-lit - 1] == 0:
                satisfied = True
        if not satisfied:
            unsat += 1
    return unsat

def heuristic_penetrance(clauses, assignment):
    total_literals = sum(len(clause) for clause in clauses)
    satisfied_literals = 0
    for clause in clauses:
        for lit in clause:
            if lit > 0 and assignment[lit - 1] == 1:
                satisfied_literals += 1
            elif lit < 0 and assignment[-lit - 1] == 0:
                satisfied_literals += 1
    return satisfied_literals / total_literals


## 5. Flip Variable

In [9]:
def flip_variable(assignment, idx):
    new_assignment = assignment.copy()
    new_assignment[idx] = 1 - new_assignment[idx]
    return new_assignment


## 6. Hill Climbing Algorithm

In [10]:
def hill_climbing(clauses, n, heuristic=heuristic_unsatisfied, max_steps=1000):
    assignment = [random.randint(0,1) for _ in range(n)]
    for step in range(max_steps):
        current_score = heuristic(clauses, assignment)
        if current_score == 0:
            return assignment
        neighbors = [flip_variable(assignment, i) for i in range(n)]
        scores = [heuristic(clauses, neighbor) for neighbor in neighbors]
        best_score = min(scores)
        if best_score >= current_score:
            return assignment
        best_idx = scores.index(best_score)
        assignment = neighbors[best_idx]
    return assignment


## 7. Beam Search Algorithm

In [11]:
def beam_search(clauses, n, beam_width=3, heuristic=heuristic_unsatisfied, max_steps=100):
    beam = [[random.randint(0,1) for _ in range(n)] for _ in range(beam_width)]
    for step in range(max_steps):
        all_neighbors = []
        for assignment in beam:
            all_neighbors.extend([flip_variable(assignment, i) for i in range(n)])
        all_neighbors = sorted(all_neighbors, key=lambda x: heuristic(clauses, x))
        beam = all_neighbors[:beam_width]
        if heuristic(clauses, beam[0]) == 0:
            return beam[0]
    return beam[0]

## 8. Variable Neighborhood Descent (VND)

In [12]:
def vnd(clauses, n, heuristic=heuristic_unsatisfied, max_steps=1000):
    assignment = [random.randint(0,1) for _ in range(n)]
    neighborhoods = [
        lambda a: [flip_variable(a, i) for i in range(n)],
        lambda a: [flip_variable(flip_variable(a, i), j)
                   for i in range(n) for j in range(i+1, n)],
        lambda a: [flip_variable(flip_variable(flip_variable(a, i), j), k)
                   for i in range(n) for j in range(i+1, n) for k in range(j+1, n)]
    ]
    for step in range(max_steps):
        improved = False
        for neighborhood in neighborhoods:
            neighbors = neighborhood(assignment)
            scores = [heuristic(clauses, neighbor) for neighbor in neighbors]
            best_score = min(scores)
            if best_score < heuristic(clauses, assignment):
                best_idx = scores.index(best_score)
                assignment = neighbors[best_idx]
                improved = True
                break
        if not improved:
            break
    return assignment

## 9. Check If Solution Satisfies Clauses

In [13]:
def is_satisfying(clauses, assignment):
    return heuristic_unsatisfied(clauses, assignment) == 0


## 10. Main Function to Test All Algorithms

In [14]:
def test_random_3sat(m, n):
    clauses = generate_k_sat(3, m, n)
    print(f"Random 3-SAT ({n} vars, {m} clauses):")
    print_clauses(clauses)

    for name, algo in [("Hill Climbing", hill_climbing),
                       ("Beam Search (width 3)", lambda c,n: beam_search(c,n,3)),
                       ("Beam Search (width 4)", lambda c,n: beam_search(c,n,4)),
                       ("VND", vnd)]:
        start = time.time()
        solution = algo(clauses, n)
        end = time.time()
        print(f"{name}:")
        print("  Solution found:" if is_satisfying(clauses, solution) else "  No complete solution")
        print(f"  Assignment: {solution}")
        print(f"  Time: {end - start:.4f}s\n")


In [15]:
if __name__ == "__main__":
    test_random_3sat(m=10, n=5)


Random 3-SAT (5 vars, 10 clauses):
x3 OR ¬x2 OR ¬x1
x1 OR x4 OR ¬x2
x5 OR ¬x3 OR ¬x2
x1 OR ¬x3 OR ¬x1
x2 OR x5 OR ¬x2
x2 OR ¬x4 OR ¬x2
x2 OR ¬x5 OR ¬x4
x1 OR x2 OR ¬x4
x1 OR x2 OR ¬x3
x1 OR x5 OR ¬x2
Hill Climbing:
  Solution found:
  Assignment: [0, 0, 0, 0, 1]
  Time: 0.0000s

Beam Search (width 3):
  Solution found:
  Assignment: [0, 1, 1, 1, 1]
  Time: 0.0001s

Beam Search (width 4):
  Solution found:
  Assignment: [1, 0, 0, 1, 0]
  Time: 0.0001s

VND:
  Solution found:
  Assignment: [0, 0, 0, 0, 1]
  Time: 0.0001s

